<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/tensorflow_2.0_docs/TensorFlow%20Core/Tutorials/Images/Image%20Segmentation/Kaggle_Nuclei_Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
!pip install --upgrade tensorflow-gpu

     |████████████████████████████████| 380.8MB 41kB/s 
     |████████████████████████████████| 450kB 44.2MB/s 
     |████████████████████████████████| 3.8MB 38.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
CPU times: user 3.19 s, sys: 656 ms, total: 3.84 s
Wall time: 54 s


In [4]:
from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle 
!chmod 600 ~/.kaggle/kaggle.json 
!kaggle competitions download -c data-science-bowl-2018

Saving kaggle.json to kaggle.json
  0% 0.00/2.62k [00:00<?, ?B/s]
100% 2.62k/2.62k [00:00<00:00, 2.25MB/s]
  0% 0.00/2.67M [00:00<?, ?B/s]
100% 2.67M/2.67M [00:00<00:00, 89.1MB/s]
 55% 5.00M/9.10M [00:00<00:00, 19.3MB/s]
100% 9.10M/9.10M [00:00<00:00, 30.4MB/s]
 94% 74.0M/79.1M [00:00<00:00, 77.2MB/s]
100% 79.1M/79.1M [00:00<00:00, 108MB/s] 
  0% 0.00/112k [00:00<?, ?B/s]
100% 112k/112k [00:00<00:00, 36.5MB/s]
100% 275M/276M [00:02<00:00, 71.4MB/s]
100% 276M/276M [00:03<00:00, 95.9MB/s]
  0% 0.00/386k [00:00<?, ?B/s]
100% 386k/386k [00:00<00:00, 49.2MB/s]


In [5]:
!ls -GFlash --color

total 368M
4.0K drwxr-xr-x 1 root 4.0K Oct 29 18:55 ./
4.0K drwxr-xr-x 1 root 4.0K Oct 29 18:53 ../
4.0K drwxr-xr-x 1 root 4.0K Oct 28 15:16 .config/
4.0K -rw-r--r-- 1 root   64 Oct 29 18:55 kaggle.json
4.0K drwxr-xr-x 1 root 4.0K Oct 25 16:58 sample_data/
4.0K -rw-r--r-- 1 root 2.7K Oct 29 18:55 stage1_sample_submission.csv.zip
388K -rw-r--r-- 1 root 387K Oct 29 18:55 stage1_solution.csv.zip
9.2M -rw-r--r-- 1 root 9.2M Oct 29 18:55 stage1_test.zip
2.7M -rw-r--r-- 1 root 2.7M Oct 29 18:55 stage1_train_labels.csv.zip
 80M -rw-r--r-- 1 root  80M Oct 29 18:55 stage1_train.zip
116K -rw-r--r-- 1 root 113K Oct 29 18:55 stage2_sample_submission_final.csv.zip
277M -rw-r--r-- 1 root 277M Oct 29 18:55 stage2_test_final.zip


In [0]:
import zipfile
files = ["stage1_train.zip","stage1_train_labels.csv.zip"]
for f in files:
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall("stage1_train")

In [10]:
!ls -GFlash --color 
!rm -rf stage1_train.zip
!rm -rf stage1_train_labels.csv.zip
!rm -rf stage1_test.zip
!rm -rf stage1_sample_submission.csv.zip
!rm -rf stage1_solution.csv.zip
!rm -rf stage2_test_final.zip
!rm -rf stage2_sample_submission_final.csv.zip

total 368M
4.0K drwxr-xr-x   1 root 4.0K Oct 29 18:56 ./
4.0K drwxr-xr-x   1 root 4.0K Oct 29 18:53 ../
4.0K drwxr-xr-x   1 root 4.0K Oct 28 15:16 .config/
4.0K -rw-r--r--   1 root   64 Oct 29 18:55 kaggle.json
4.0K drwxr-xr-x   1 root 4.0K Oct 25 16:58 sample_data/
4.0K -rw-r--r--   1 root 2.7K Oct 29 18:55 stage1_sample_submission.csv.zip
388K -rw-r--r--   1 root 387K Oct 29 18:55 stage1_solution.csv.zip
9.2M -rw-r--r--   1 root 9.2M Oct 29 18:55 stage1_test.zip
 76K drwxr-xr-x 672 root  72K Oct 29 18:57 stage1_train/
2.7M -rw-r--r--   1 root 2.7M Oct 29 18:55 stage1_train_labels.csv.zip
 80M -rw-r--r--   1 root  80M Oct 29 18:55 stage1_train.zip
116K -rw-r--r--   1 root 113K Oct 29 18:55 stage2_sample_submission_final.csv.zip
277M -rw-r--r--   1 root 277M Oct 29 18:55 stage2_test_final.zip


In [27]:
%%time
import os
import sys
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow
from cv2 import resize
import matplotlib.pyplot as plt
%matplotlib inline

CPU times: user 1.13 ms, sys: 0 ns, total: 1.13 ms
Wall time: 1.2 ms


In [0]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

DATA_PATH = 'stage1_train/'
seed = 2019

In [0]:
np.random.seed(seed)
tf.random.set_seed(seed)

In [0]:
image_ids = next(os.walk(DATA_PATH))[1]

In [0]:
X = np.zeros((len(image_ids),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS), dtype=np.uint8)
Y = np.zeros((len(image_ids),IMG_HEIGHT, IMG_WIDTH,1),dtype=np.bool)

In [44]:
for n, id_ in tqdm(enumerate(image_ids),total=len(image_ids)):
    path = DATA_PATH + id_
    img = imread(path+'/images/'+id_+'.png')[:,:,:IMG_CHANNELS]
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH))
    X[n] = img
    mask = np.zeros((IMG_HEIGHT,IMG_WIDTH,1),dtype=np.bool)
    for mask_file in next(os.walk(path+'/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH)), axis=-1)
        mask = np.maximum(mask, mask_)
    Y[n] = mask

100%|██████████| 670/670 [00:42<00:00, 15.62it/s]


In [0]:
x_train = X
y_train = Y

![unet](https://i1.wp.com/androidkt.com/wp-content/uploads/2019/04/Selection_062.png?resize=768%2C561&ssl=1)